# Initial Exploring and Cleaning of Data Set
### Define Functions for Load, Reformatting, and Basic Summarization Tasks

In [1]:
import pandas as pd
from datetime import datetime
import pytz

pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [2]:
def dataImport(file):
    '''
    Imports the data
    Inputs: file (str)
    Outputs: DF (Pandas DF)
    '''
    #import the dataset
    DF = pd.read_csv(file)
    
    return DF

In [3]:
def summaryStats(DF):
    '''
    Generate Summmary Statitics for the full feature space
    Input: DF (pd DF)
    Output: None
    '''
    
    print('\n###########', 'Summary Statistics', '###########\n')
    print('Dataframe Structure:', DF.shape)

    print('\nVariable Types:')
    print(DF.dtypes)
    
    print('\nSummary Stats (Quantitative Features)')
    print(DF.describe())
    
    print('\nSummary Stats (Qualitative Features)')
    print(DF.describe(include=['object']))

In [4]:
def reformatData(DF):
    '''
    Restructure the data as required (change column names, dtypes, etc) 
    Input: DF (pd DF)
    Output: textDF (pd DF)
    '''
    #rename the columns
    DF.columns = ['Datetime', 'TweetID','FullText', 'Truncated', 'RepliedTo',
                  'GeoTagged', 'Coordinates', 'Location', 'RetweetCount', 
                  'FavoriteCount', 'Sensitive?', 'Lang', 'UserName', 'UserLocation',
                  'UserDescription', 'UserFollowers', 'UserFriends', 'UserStatusCount']
    
    #convert the columns to the appropriate data types
    DF['Datetime']= pd.to_datetime(DF['Datetime']) 
    
    #convert remaining columns to the appropriate type
    DF = DF.astype({'Sensitive?': bool, 'TweetID': str})
    
    #only focused on the text data so only retain columns of interest for
    #textual analysis
    textDF = DF[['Datetime', 'TweetID', 'FullText', 'UserName', 'Lang']]
        
    return textDF

In [5]:
def scoreData(DF):
    '''
    Score the dataframe to ensure that each feature is in the appropriate value range
    and/or has no NA
    Input: DF (pd DF)
    Output: None
    '''
    ##check datetime range
    bad_date = 0
    
    #check date range (June 1, 2019 to Super Tuesday 2020)
    for date in DF['Datetime']:
        if date < datetime(2019, 6, 1, 0, 0, 0, 0, pytz.UTC) or \
            date > datetime(2020, 3, 3, 0, 0, 0, 0, pytz.UTC):
            bad_date += 1
    
    #check for null values
    bad_date += sum(DF['Datetime'].isnull())
    
    #TweetID should be unique
    bad_id = len(DF['TweetID'])-len(DF['TweetID'].drop_duplicates())
    bad_id += sum(DF['TweetID'].isnull())
    
    #FullText cannot be empty
    bad_text = sum(DF['FullText'].isnull()) 
    
    #UserName
    names = ['Amy Klobuchar', 'Bernie Sanders', 'Elizabeth Warren', 'Joe Biden',
             'Pete Buttigieg', 'Tom Steyer', 'Tulsi Gabbard 🌺']
    
    bad_names = 0
    
    for name in DF['UserName']:
        if name not in names:
            bad_names += 1
     
    ## check if the language is not english            
    bad_lang = 0
    
    for language in DF['Lang']:
        if language != 'en':
            bad_lang += 1
    
    ## generate the score
    print('\n######## Data Frame Score by Column #########')   
    print("Date Column: ", bad_date)
    print("TweetID Column: ", bad_id)
    print("Text Column: ", bad_text)
    print("UserNames Column:", bad_names)
    print("Language Column (not English):", bad_lang)
    
    bad_total = bad_date + bad_id + bad_text + bad_names + bad_lang
    score = bad_total / (DF.shape[0]*DF.shape[1])
    
    print('\n######## Total DF Score #########')   
    print('Total DF Cleanliness Score:', 100-round(score * 100, 2))

In [6]:
def cleanData(DF):
    '''
    Clean the dataset by dropping duplicate tweets and removing those that 
    are not in english
    Input: DF (pd DF)
    Output: DF (pd DF)
    '''
    #drop columns with duplicate tweet ID
    DF = DF.drop_duplicates('TweetID', keep = 'last')

    #drop tweets that are not in English    
    DF = DF[DF['Lang'] == 'en']
    
    return DF

In [7]:
def scoreData(DF):
    '''
    Score the dataframe to ensure that each feature is in the appropriate value range
    and/or has no NA
    Input: DF (pd DF)
    Output: None
    '''
    ##check datetime range
    bad_date = 0
    
    #check date range (June 1, 2019 to Super Tuesday 2020)
    for date in DF['Datetime']:
        if date < datetime(2019, 6, 1, 0, 0, 0, 0, pytz.UTC) or \
            date > datetime(2020, 3, 3, 0, 0, 0, 0, pytz.UTC):
            bad_date += 1
    
    #check for null values
    bad_date += sum(DF['Datetime'].isnull())
    
    #TweetID should be unique
    bad_id = len(DF['TweetID'])-len(DF['TweetID'].drop_duplicates())
    bad_id += sum(DF['TweetID'].isnull())
    
    #FullText cannot be empty
    bad_text = sum(DF['FullText'].isnull()) 
    
    #UserName
    names = ['Amy Klobuchar', 'Bernie Sanders', 'Elizabeth Warren', 'Joe Biden',
             'Pete Buttigieg', 'Tom Steyer', 'Tulsi Gabbard 🌺']
    
    bad_names = 0
    
    for name in DF['UserName']:
        if name not in names:
            bad_names += 1
     
    ## check if the language is not english            
    bad_lang = 0
    
    for language in DF['Lang']:
        if language != 'en':
            bad_lang += 1
    
    ## generate the score
    print('\n######## Data Frame Score by Column #########')   
    print("Date Column: ", bad_date)
    print("TweetID Column: ", bad_id)
    print("Text Column: ", bad_text)
    print("UserNames Column:", bad_names)
    print("Language Column (not English):", bad_lang)
    
    bad_total = bad_date + bad_id + bad_text + bad_names + bad_lang
    score = bad_total / (DF.shape[0]*DF.shape[1])
    
    print('\n######## Total DF Score #########')   
    print('Total DF Cleanliness Score:', 100-round(score * 100, 2))

### _Load and Understand the Data_

In [8]:
DF = dataImport('Data/DemTweetsReformat.csv')
DF.head(2)

,created_at,id,full_text,truncated,in_reply_to_screen_name,geo,coordinates,place,retweet_count,favorite_count,possibly_sensitive,lang,user.name,user.location,user.description,user.followers_count,user.friends_count,user.statuses_count
0,Mon Mar 02 01:07:24 +0000 2020,1234284135386095616,"John Lewis leading marchers across the Edmund Pettus Bridge in Selma in 1965. What an honor to see him there today, fighting cancer but his voice just as strong. https://t.co/UMp7Snao7T",False,NaN,NaN,NaN,NaN,599,4156,False,en,Amy Klobuchar,NaN,U.S. Senator from Minnesota. Text AMY to 91990 for updates.,1066377,137980,11782
1,Mon Mar 02 00:56:17 +0000 2020,1234281337776955392,Pete Buttigieg has run an inspiring and historic campaign. I have so much respect for you @PeteButtigieg and know there are great things ahead. And both John and I are big fans of Chasten!,False,NaN,NaN,NaN,NaN,5019,89534,NaN,en,Amy Klobuchar,NaN,U.S. Senator from Minnesota. Text AMY to 91990 for updates.,1066377,137980,11782


In [9]:
## get the number of tweets by user
print(DF['user.name'].value_counts())

Elizabeth Warren    2347
Joe Biden           2183
Amy Klobuchar       2017
Tom Steyer          1990
Bernie Sanders      1861
Pete Buttigieg      1700
Tulsi Gabbard 🌺     1085
Name: user.name, dtype: int64


In [10]:
## generate basic summary statistics
summaryStats(DF)


########### Summary Statistics ###########

Dataframe Structure: (13183, 18)

Variable Types:
created_at                  object
id                           int64
full_text                   object
truncated                     bool
in_reply_to_screen_name     object
geo                        float64
coordinates                float64
place                      float64
retweet_count                int64
favorite_count               int64
possibly_sensitive          object
lang                        object
user.name                   object
user.location               object
user.description            object
user.followers_count         int64
user.friends_count           int64
user.statuses_count          int64
dtype: object

Summary Stats (Quantitative Features)
                 id  geo  coordinates  place  retweet_count  favorite_count  \
count  1.318300e+04  0.0          0.0    0.0   13183.000000    13183.000000   
mean   1.196794e+18  NaN          NaN    NaN    1385.434499     

In [11]:
## reformat the dataframe--removing unneeded column
## modifying data types, etc.
DF = reformatData(DF)

In [12]:
## get a preliminary score for the dataset cleanliness
scoreData(DF)


######## Data Frame Score by Column #########
Date Column:  0
TweetID Column:  138
Text Column:  0
UserNames Column: 0
Language Column (not English): 257

######## Total DF Score #########
Total DF Cleanliness Score: 99.4


In [13]:
## clean the data
DF = cleanData(DF)

## rescore the data
scoreData(DF)


######## Data Frame Score by Column #########
Date Column:  0
TweetID Column:  0
Text Column:  0
UserNames Column: 0
Language Column (not English): 0

######## Total DF Score #########
Total DF Cleanliness Score: 100.0


In [14]:
## write cleaned dataset to Data folder
DF.to_csv('Data/CleanedTweets.csv', index=False)